In [ ]:
import numpy as np
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.colors as colors
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d


import pandas as pd
import scipy
import scanpy as sc

from sklearn import datasets
from sklearn.decomposition import PCA

from numba import jit

import celltypist
from celltypist import models

from matplotlib.cm import ScalarMappable

In [ ]:
#Custom colormap

from matplotlib.cm import register_cmap
from matplotlib.colors import ListedColormap

tab20b = matplotlib.colormaps['tab20b']
tab20c = matplotlib.colormaps['tab20c']
colors1 = tab20b(np.linspace(3.001/5., 1, 8))
colors2 = tab20c(np.linspace(0, 3.999/5., 16))

colors = np.concatenate([colors1, colors2])

map_name = 'op_tab24'
op_cmap = ListedColormap(colors, name=map_name )
matplotlib.colormaps.register(name=map_name, cmap=op_cmap)

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid

def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap


In [ ]:
gspDF = pd.read_csv('/Users/oipulk/Documents/prime_data/preprocessed/gsp_clin_noiRECISTNans.csv',low_memory=False, index_col=0)
pspDF = pd.read_csv('/Users/oipulk/Documents/prime_data/preprocessed/psp_clin_noiRECISTNans.csv',low_memory=False, index_col=0)

gsp_genes = gspDF.index.values[13:len(gspDF.index.values)]
psp_genes = pspDF.index.values[13:len(pspDF.index.values)]
ctl_genes = ['CD8A', 'CD8B', 'GZMA', 'GZMB', 'PRF1']

In [ ]:
## Genes tested in laboratory and their p-value in t-test for response

orderedGenes = np.array(['SLAMF7', 'ATP2A3', 'GBP1', 'HSH2D', 'LCK', 'MAP4K1', 'CSF2RB', 'CD247', 'NCF1', 'LCP2', 'PRKCH', 'ITGAL', 'LY9', 'HLA-DPA1', 'TIGIT', 'CCDC69', 'RASAL3', 'IL12RB1', 'SASH3', 'IL2RG', 'ARHGAP9', 'ARHGAP25', 'PTK2B', 'TYMP', 'PSD4', 'WAS', 'CD27', 'SLAMF6', 'TMC8', 'GBP5', 'SEPT1', 'TRAF3IP3', 'PSMB10', 'GZMB', 'CD3E', 'SYK', 'PSTPIP1', 'GNLY', 'MYO1G', 'CD40', 'POU2F2', 'CD38', 'SEMA4D', 'FCRL5', 'IGFLR1', 'SIRPG', 'BTK', 'ITGB2', 'HCLS1', 'CD48', 'AOAH', 'RCSD1', 'GPR174', 'SEMA4A', 'PARVG', 'PRF1', 'DOCK2', 'FGR', 'IL10RA', 'FAM107B', 'EVL', 'BIN2', 'HLA-DOB', 'CD52', 'LST1', 'CD84', 'CD53', 'PIK3AP1', 'CLEC2D', 'LYN', 'PTPRC', 'CD3D', 'CD96', 'DOK2', 'TRIM22', 'FGD2', 'EPSTI1', 'CD19', 'VNN2', 'PIK3CG', 'CASP10', 'CD79A', 'GZMA', 'CD2', 'HLA-F', 'MZB1', 'ERAP1', 'CD37', 'FERMT3', 'PLCG2', 'CCR5', 'BTN3A3', 'ZAP70', 'SELL', 'HAVCR2', 'FLT3LG', 'CTSS', 'TRAT1', 'TNFAIP2', 'BLNK', 'ARHGAP30', 'TTC7A', 'SPN', 'CTLA4', 'GIMAP7', 'GBP3', 'LCP1', 'TAPBP', 'CLIC2', 'CYTH4', 'SLC7A7', 'TNFRSF1B', 'BTN3A1', 'LILRB2', 'SEPT6', 'LAT2', 'CCR2', 'CD4', 'NFAM1', 'TAPBPL', 'IGSF6', 'SECTM1', 'UBA7', 'IDO1', 'RSAD2', 'CYBB', 'PTAFR', 'PLCB2', 'ADAM28', 'SELPLG', 'TMEM176B', 'PTPN22', 'NAAA', 'HCK', 'HLA-DQB1', 'RASGEF1B', 'TBXAS1', 'LSP1', 'ADAM8', 'MPEG1', 'TNFSF10', 'PLD4', 'LILRB4', 'STK17B', 'ARRB2', 'SLCO2B1', 'TNFSF13', 'SLAMF1', 'CD28', 'SLC15A3', 'ITGAM', 'RASGRP2', 'VCAM1', 'CLEC7A', 'HLA-DMB', 'CTSW', 'HCST', 'SLC8A1', 'SKAP1', 'IL2RB', 'BASP1', 'RNASE6', 'LGMN'
,'PIK3IP1', 'CYTH1', 'ALDH2', 'GLIPR1', 'ELMO1', 'EVI2B', 'MFNG', 'VAMP5', 'LILRA4', 'MAN1A1', 'KLHL6', 'CD1C', 'CD163', 'MS4A6A', 'ICAM2', 'ACE', 'FGL2', 'NCKAP1L', 'KCNMA1', 'SRGN', 'CST7', 'DAPP1', 'FOLR2', 'ENTPD1', 'PILRA', 'HLA-DMA', 'SAMSN1', 'LAMP3', 'CD14', 'PDCD1LG2', 'CLEC10A', 'GNA15', 'ADAP2', 'SPINT2', 'CD33', 'CR1', 'CXCL12', 'IL7R', 'AIF1', 'PTGER4', 'EMB', 'FAM57A', 'SLC39A10'])


orderedGenes_p = np.array([0.000002, 0.000171, 0.00026, 0.000262, 0.000288, 0.000316, 0.000355, 0.000377, 0.000445, 0.00051, 0.000515, 0.000672, 0.000698, 0.000757, 0.000763, 0.000841, 0.000858, 0.000889, 0.000987, 0.00131, 0.00138, 0.001427, 0.001449, 0.001493, 0.001516, 0.001584, 0.001614, 0.001712, 0.001857, 0.00186, 0.001904, 0.002429, 0.002451, 0.002531, 0.002572, 0.002623, 0.002647, 0.002652, 0.002658, 0.002798, 0.003069, 0.00316, 0.003734, 0.003758, 0.004254, 0.00434, 0.004603, 0.004735, 0.004918, 0.005094, 0.005575, 0.005847, 0.006044, 0.006048, 0.006177, 0.006216, 0.006241, 0.006269, 0.00664, 0.006877, 0.007022, 0.007058, 0.007183, 0.007487, 0.008256, 0.008742, 0.008766, 0.008771, 0.009672, 0.009696, 0.009935, 0.01011, 0.010196, 0.010751, 0.011399, 0.011402, 0.011679, 0.011761, 0.012181, 0.012613, 0.012746, 0.012924, 0.012959, 0.013217, 0.013969, 0.014275, 0.015414, 0.015982, 0.017934, 0.018331, 0.018413, 0.018533, 0.019213, 0.020528, 0.020969, 0.021716, 0.022919, 0.024311, 0.02618, 0.026245, 0.026588, 0.027259, 0.027294, 0.027622, 0.027867, 0.027916, 0.029668, 0.030289, 0.030526, 0.031287, 0.031835, 0.032011
, 0.032486, 0.033289, 0.033758, 0.034644, 0.034706, 0.03764, 0.038836, 0.0441, 0.044157, 0.049343, 0.053912, 0.058147, 0.058859, 0.05891, 0.063574, 0.064643, 0.064875, 0.070647, 0.07081, 0.071064, 0.071227, 0.077519, 0.080847, 0.083266, 0.085183, 0.091863, 0.100711, 0.102939, 0.105291, 0.110201, 0.119018, 0.125466, 0.127341, 0.140282, 0.143816, 0.146222, 0.149913, 0.15029, 0.151847, 0.162302, 0.162963, 0.162976, 0.166283, 0.1701, 0.17621, 0.179118, 0.199627, 0.202331, 0.220066, 0.243455, 0.244815, 0.24494, 0.250183, 0.268815, 0.270333, 0.270414, 0.283139, 0.285096, 0.300669, 0.304284, 0.319236, 0.329626, 0.347148, 0.356216, 0.362608, 0.365105, 0.368516, 0.387825, 0.388237, 0.412144, 0.414301, 0.41613, 0.468657, 0.491555, 0.51177, 0.54486, 0.564074, 0.5938, 0.60468, 0.615798, 0.636671, 0.65277, 0.657607, 0.661587, 0.701709, 0.721311, 0.756649, 0.778583, 0.819239, 0.837958, 0.86186, 0.955137, 0.001639, 0.068734])

orderedGenes = orderedGenes[np.argsort(orderedGenes_p)]
orderedGenes_p = np.sort(orderedGenes_p)

In [ ]:
adata= sc.read_loom(
    "/Users/oipulk/Documents/scRNASeq/data/Li2018/Li.loom")

In [ ]:
# Cell type predictions by Celltypist
predictions = celltypist.annotate(adata, model = 'Immune_All_Low.pkl', majority_voting = True)
print(predictions.predicted_labels)
adata = predictions.to_adata()

In [ ]:
# Compute frequencies of anti-MAA target counts in each patient (normalized by total number of target annotations, and all cell counts)
patients = np.unique(adata.obs['patient'].values)
anti_maa_f = np.zeros(adata.shape[0])
anti_maa_f_all = np.zeros(adata.shape[0])

for pat in patients:

    if (sum(adata.obs['patient']==pat)!=sum(adata[adata.obs['patient']==pat].obs['target.y']=='NA')):
        
        prop = sum(adata[adata.obs['patient']==pat].obs['target.y']=='anti-melanoma')/(sum(adata.obs['patient']==pat)-sum(adata[adata.obs['patient']==pat].obs['target.y']=='NA'))
    
    else:

        prop=np.nan
    
    prop_all = sum(adata[adata.obs['patient']==pat].obs['target.y']=='anti-melanoma')/sum(adata.obs['patient']==pat)
    anti_maa_f[np.where(adata.obs['patient']==pat)[0]] = prop
    anti_maa_f_all[np.where(adata.obs['patient']==pat)[0]] = prop_all

adata.obs['anti_maa_f'] = anti_maa_f
adata.obs['anti_maa_f_all'] = anti_maa_f_all

In [ ]:
# Compute CTL score in each patient
ctl_genes = ['CD8A', 'CD8B', 'GZMA', 'GZMB', 'PRF1']
sc.tl.score_genes(adata, ctl_genes, score_name='ctl_score')

pt_ctl_scores = np.zeros(adata.shape[0])
for pat in patients:

    score = np.nanmean(adata[adata.obs['patient']==pat].obs['ctl_score'])
    pt_ctl_scores[np.where(adata.obs['patient']==pat)[0]] = score

adata.obs['ctl_score_pt'] = pt_ctl_scores

In [ ]:
# Add an indicator for prior immunotherapy
IT = (adata.obs['treatment'].values=='1IT')|(adata.obs['treatment'].values=='2IT')|(adata.obs['treatment'].values=='mIT').astype(int)
adata.obs['IT'] = IT 
adata.obs['IT'] = adata.obs['IT'].astype('category')


In [ ]:
# Therapy_naive patients
adata_th_naive = adata[adata.obs['IT']==0].copy()

# Normalize counts log-transform again
sc.pp.normalize_total(adata_th_naive, target_sum=1e4)
sc.pp.log1p(adata_th_naive)
print(np.expm1(adata_th_naive.X).sum(axis=1))

In [ ]:
# Compute umap and t-sne
sc.pp.pca(adata_th_naive)
sc.pp.neighbors(adata_th_naive)
sc.tl.umap(adata_th_naive)
sc.tl.tsne(adata_th_naive)
umap_coordinates = adata_th_naive.obsm['X_umap']

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, dpi_save=1200, frameon=True, vector_friendly=True, fontsize=8,
                         figsize=(9,8),  format='pdf', facecolor=None, transparent=False, ipython_format='png2x')

In [ ]:
# First check the current categories
print("Current categories:", adata_th_naive.obs['majority_voting'].cat.categories)

# ALveolar macrophages is wrong -> Suppressor macrophages (annotation by ACT based on DEGs)
adata_th_naive.obs['majority_voting'] = adata_th_naive.obs['majority_voting'].cat.rename_categories({
    'Alveolar macrophages': 'Suppressor macrophages'
})

# Verify the changes
print("Updated categories:", adata_th_naive.obs['majority_voting'].cat.categories)


In [ ]:
sc.pl.tsne(adata_th_naive, color = 'majority_voting', palette='op_tab24', legend_loc = 'on data',save='Annotations_all_types.pdf' )

In [ ]:
sc.pl.tsne(adata_th_naive, color = 'majority_voting', palette='op_tab24', legend_loc = 'right margin',save='Annotations_all_types_legend_right.pdf' )

## Analysis of monocytes, macrophages, DCs 

In [ ]:
# Find communities of cells (cell types)
sc.tl.leiden(adata_th_naive, key_added='leiden_initial')
# sc.tl.louvain(adata_th_naive)

In [ ]:
# Plot UMAP with colors for each cell type by 'leiden' community finding algorithm
sc.set_figure_params(scanpy=True, dpi=300, dpi_save=1200, frameon=True, vector_friendly=True, fontsize=8,
                         figsize=(9,8),  format='pdf', facecolor=None, transparent=False, ipython_format='png2x')

sc.pl.tsne(adata_th_naive, color='leiden_initial', palette='op_tab24', legend_loc='on data')

In [ ]:
# 1. Iitial clustering

# 2. Identify the cluster(s) containing DC2, monocytes, and macrophages
target_clusters = ['6','8','13']  # Replace with your actual cluster IDs

# 3. Subset the data
adata_subset = adata_th_naive[adata_th_naive.obs['leiden_initial'].isin(target_clusters)].copy()

# 4. Recompute the neighborhood graph on the subset
sc.pp.neighbors(adata_subset)

# 5. Perform Leiden clustering at higher resolution on the subset
sc.tl.leiden(adata_subset, resolution=0.75, key_added='leiden_refined')

# 6. Prepare categories for the combined clustering
initial_categories = list(adata_th_naive.obs['leiden_initial'].cat.categories)
refined_categories = list(adata_subset.obs['leiden_refined'].cat.categories)

# Remove target clusters from initial categories
initial_categories_filtered = [cat for cat in initial_categories if cat not in target_clusters]

# Create new category names for refined clusters
refined_categories_renamed = [f'r{cat}' for cat in refined_categories]

# Combine filtered initial categories with renamed refined categories
combined_categories = initial_categories_filtered + refined_categories_renamed

# 7. Create new column for combined clustering
adata_th_naive.obs['leiden'] = pd.Categorical(
    adata_th_naive.obs['leiden_initial'],
    categories=combined_categories
)

# 8. Update the combined clustering for the refined subset
for idx in adata_subset.obs.index:
    refined_value = adata_subset.obs.loc[idx, 'leiden_refined']
    adata_th_naive.obs.at[idx, 'leiden'] = f'r{refined_value}'

# 9. Optionally, sort the categories for better readability
adata_th_naive.obs['leiden'] = adata_th_naive.obs['leiden'].cat.reorder_categories(sorted(adata_th_naive.obs['leiden'].cat.categories))

In [ ]:
sc.pl.tsne(adata_th_naive, color='leiden', palette='op_tab24', legend_loc='on data')

In [ ]:
# Check that all clusters express either CD45 or CD3 (i.e. they are immune cells)
sc.pl.tsne(adata_th_naive, color=['CD45','CD3'],palette='tab20',cmap='coolwarm',vmax=2.0)


In [ ]:
adata_subset = adata_th_naive[adata_th_naive.obs['leiden'].isin(['r0','r1','r2','r3','r4','r5','r6','r7','r8','16','17'])].copy()

In [ ]:
# Cell type predictions by Celltypist
predictions_subset = celltypist.annotate(adata_subset, model = 'Immune_All_High.pkl', majority_voting = True)
print(predictions_subset.predicted_labels)
adata_subset = predictions_subset.to_adata()

In [ ]:
sc.pl.tsne(adata_subset, color = 'majority_voting', palette='tab20',legend_loc = 'right margin' )

In [ ]:
# Markers from ACT human pan-tissue (frequency>5)
pDC_markers = ['IL3RA','LILRA4','PLD4']
sc.tl.score_genes(adata_subset, pDC_markers, score_name='pDC_score')
sc.pl.tsne(adata_subset, color=['pDC_score'],palette='tab20',cmap='coolwarm',vmax=2)

# cDC1_markers = ['CLEC9A','XCR1','BATF3','CADM1','CD1C'
# ,'IDO1','ANXA6','CD40','CLIC2','CST7','ETV3','ID2','IL6ST','NET1','PTDSS1'
# ,'SCARB1','TNFRSF10B']

cDC1_markers = ['CLEC9A','XCR1','BATF3','IRF8']

sc.tl.score_genes(adata_subset, cDC1_markers, score_name='cDC1_score')
sc.pl.tsne(adata_subset, color=['cDC1_score'],palette='tab20',cmap='coolwarm',vmax=1.0)

# cDC2_markers = ['CD1C','CLEC10A','FCER1A','SIRPA','ADAM8','ADAM28','ARAF','AXL','FCGR2B','HMGA1','IRF4','LY86','PFDN1','TIMM13']

cDC2_markers = ['CD1C','CLEC10A','IRF4', 'NOTCH2']

sc.tl.score_genes(adata_subset, cDC2_markers, score_name='cDC2_score')
sc.pl.tsne(adata_subset, color=['cDC2_score'],palette='tab20',cmap='coolwarm',vmax=1.5)

#Langerhans cells
LC_markers = ['CD207', 'CD1A', 'CDH1', 'EPCAM']
sc.tl.score_genes(adata_subset, LC_markers, score_name='LC_score')
sc.pl.tsne(adata_subset, color=['LC_score'],palette='tab20',cmap='coolwarm',vmax=1.5)

migratory_state_markers = ['CCR7', 'LAMP3','HLA-DRA']
sc.tl.score_genes(adata_subset, migratory_state_markers, score_name='migratory_state_score')
sc.pl.tsne(adata_subset, color=['migratory_state_score'],palette='tab20',cmap='coolwarm',vmax=3.5)


In [ ]:
#individual markers
sc.pl.tsne(adata_subset, color=['CD14','SIRPA','XCR1'],palette='tab20',cmap='coolwarm')
sc.pl.tsne(adata_subset, color=['CD40', 'CD80', 'CD86'],palette='tab20',cmap='coolwarm')


In [ ]:
sc.tl.rank_genes_groups(adata_th_naive,'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata_th_naive, n_genes=20, sharey=False)

de_markers = sc.get.rank_genes_groups_df(adata_th_naive, None)
de_markers = de_markers[(de_markers.pvals_adj < 0.03) & (de_markers.logfoldchanges > 1.0)]
de_markers

In [ ]:
# Give cluster name to see DEGs compared to other clusters
for cluster in ['r0','r1','r2','r3','r4','r5','r6','r7','r8','16','17']:
    print(cluster,':'+', '.join(de_markers[de_markers['group']==cluster].iloc[np.argsort(de_markers[de_markers['group']==cluster]['scores'])[::-1],:][0:50].names.values))


### Compute PRIME scores of cells and patients

In [ ]:
# Load Prime score 
PRIME_df= pd.read_csv('/Users/oipulk/Documents/prime_v3/MS/pub_tables/bDEA_results_and_PRIME_weights.csv', index_col=0)

In [ ]:
PRIME_df.head(20)

In [ ]:
#LAP3 (or its aliases) is not in the data
PRIME_weights = np.append(PRIME_df.weights.values[0:9],PRIME_df.weights.values[10:15])
PRIME_genes = np.append(PRIME_df.index[0:9],PRIME_df.index[10:15])

In [ ]:
# Check the correct format: PRIME uses log2-transformed TPM data
print(np.expm1(adata_th_naive.X).sum(axis=1))

In [ ]:
xpr_data = adata_th_naive.copy()

## Here we do not need to take the log again
# PRIME_score = np.asarray(np.matmul(PRIME_weights,np.log2(1+xpr_data[:,PRIME_genes].X.todense()).T))[0]
PRIME_score = np.asarray(np.matmul(PRIME_weights,xpr_data[:,PRIME_genes].X.todense().T))[0]

adata_th_naive.obs['PRIME_score'] = PRIME_score

In [ ]:
pt_prime_scores = np.zeros(adata_th_naive.shape[0])
for pat in patients:

    score = np.nanmean(adata_th_naive[adata_th_naive.obs['patient']==pat].obs['PRIME_score'])
    pt_prime_scores[np.where(adata_th_naive.obs['patient']==pat)[0]] = score

adata_th_naive.obs['PRIME_score_pt'] = pt_prime_scores

## PRIME, CD74, SLAMF7 and TYMP expression

In [ ]:
sc.pl.tsne(adata_th_naive, color = 'CD74', cmap = 'coolwarm', save='_CD74.pdf', vmax=6 )

In [ ]:
sc.pl.tsne(adata_th_naive, color = 'SLAMF7', cmap = 'coolwarm', save='_SLAMF7.pdf', vmax=3 )

In [ ]:
sc.pl.tsne(adata_th_naive, color = 'TYMP', cmap = 'coolwarm', save='_TYMP.pdf', vmax=3 )

In [ ]:
sc.pl.tsne(adata_th_naive, color = 'PRIME_score', cmap = 'coolwarm', save='_PRIME_score.pdf', vmax=1 )

# DEA

In [ ]:
import os
os.getcwd()


In [ ]:
## Import results of bulk DEA based on tximport and DESeq2
bDEA_res = pd.read_csv('./data/DEA_results_3cohorts.csv', index_col=0)

## DEA: pDC, DC, migratory DC vs Rest in the monocytes cluster

In [ ]:
DC_sub = ['pDC','DC','Migratory DCs']
monocytes_rest = ['Erythrophagocytic macrophages',
 'Macrophages',
 'Intermediate macrophages',
 'Alveolar macrophages',
 'DC2',
 'Non-classical monocytes',
 'Classical monocytes']

monocyte_cl = ['Erythrophagocytic macrophages','Macrophages', 'Intermediate macrophages', 'Alveolar macrophages',
               'pDC', 'DC', 'DC2','Migratory DCs',
               'Non-classical monocytes','Classical monocytes']

adata_monocytes_pre = adata_th_naive[adata_th_naive.obs['majority_voting'].isin(monocyte_cl)]

# Define a categorical variable for a cell belonging to the DC sub cluster
adata_monocytes_pre.obs['DC_sub'] = adata_monocytes_pre.obs['majority_voting'].isin(DC_sub).astype('category')

In [ ]:
sc.tl.rank_genes_groups(adata_monocytes_pre, groupby ='DC_sub', method='wilcoxon', key_added = "DC_subCl_vs_other_monocytes_pre_wilcoxon")
sc.pl.rank_genes_groups(adata_monocytes_pre, n_genes=25, sharey=False, key="DC_subCl_vs_other_monocytes_pre_wilcoxon", save='DC_subCl_vs_other_monocytes_pre_wilcoxon.pdf')

In [ ]:
scDEA_res = adata_monocytes_pre.uns['DC_subCl_vs_other_monocytes_pre_wilcoxon']
groups = scDEA_res['names'].dtype.names
group = 'True'

sc_pvals = scDEA_res['pvals_adj'][group]
sc_sig_genes_mask = (sc_pvals < 0.05)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# See if there are sc-DEG's that are not part of the bulk-DEA 
np.setdiff1d(sc_sig_gene_names,np.intersect1d(sc_sig_gene_names,scDEA_res['names'][group]))

In [ ]:
de_markers = sc.get.rank_genes_groups_df(adata_monocytes_pre, group='True', key='DC_subCl_vs_other_monocytes_pre_wilcoxon')
de_markers = de_markers[(de_markers.pvals_adj < 0.03) & (np.abs(de_markers.logfoldchanges) > 1.0)]
de_markers.to_csv('./data/scDEA_DC_results_padj003_lfc1.csv', index=False)

In [ ]:
de_markers

In [ ]:
#Volcano plot

## Uncomment these to define a new cmap (run on 1st time)
orig_cmap = matplotlib.cm.RdBu
cmap_DC = shiftedColorMap(orig_cmap, start=0., midpoint=0.52, name='cmap_DC_RdBu052')

# Select genes to be labelled: either significant in t-test for response or intersection with DEG's from bulk DEA for response
# sig_in_bDEA_and_tpm = np.intersect1d(orderedGenes[orderedGenes_p<0.05],bDEA_res[bDEA_res['pvalue']<0.05].index.values)
sig_in_tpm = orderedGenes[orderedGenes_p<0.05]

scDEA_res = adata_monocytes_pre.uns['DC_subCl_vs_other_monocytes_pre_wilcoxon']

# Get the scores, p-values and log fold changes
sc_scores = scDEA_res['scores'][group]
sc_pvals = scDEA_res['pvals_adj'][group]
sc_logfoldchanges = scDEA_res['logfoldchanges'][group]

# Show only genes that pass a significance threshold (e.g., adjusted p-value < 0.05)
sc_sig_genes_mask = (sc_pvals < 0.5)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# Restrict to genes included in both analyses
sc_sig_gene_names = np.intersect1d(sc_sig_gene_names, bDEA_res.index)

# Scores, p-values and LFCs in DEA for cell type
sc_sig_scores_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_pvals_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_logfoldchanges_ordered = np.zeros(len(sc_sig_gene_names))

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_scores_ordered[i] = sc_scores[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_pvals_ordered[i] = sc_pvals[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_logfoldchanges_ordered[i] = sc_logfoldchanges[[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]]


sc_sig_neg_log_pvals_ordered = -np.log(sc_sig_pvals_ordered)

sc_sig_p_from_bDEA = np.array([])
sc_sig_LFC_from_bDEA = np.array([])

#Collect names of genes that are significantly DE in both Sc and bulk data
sig_sc_bulk_DC = np.array([])

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_p_from_bDEA = np.append(sc_sig_p_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['padj'])
    sc_sig_LFC_from_bDEA = np.append(sc_sig_LFC_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['log2FoldChange'])
    
# Create the volcano plot
fig, ax = plt.subplots(figsize=(6, 4))

bDEA_p_norm = -np.log(sc_sig_p_from_bDEA)/np.nanmax(-np.log(sc_sig_p_from_bDEA))

# Define color scales for scatter plot
dotcolors = sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]
ringcolors = (1+np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.max(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.+(1-np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.min(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.

# Scatter plot
ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), facecolor='none', c=ringcolors, cmap=cmap_DC, s=4+52.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=10,alpha=0.9)
scatter = ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), c=dotcolors, cmap=cmap_DC, s=2+50.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=15,alpha=0.9)
ax.grid(False)

# Plot colorbar
cbar = fig.colorbar(scatter, shrink=0.6, pad=0.02)
cbar.ax.tick_params(labelsize=7)
cbar.set_label(label='Bulk DEA '+r'$\mathrm{log}_2$-fold change for response',size=7, labelpad=0.3)

labeled_genes_DC = np.intersect1d(sig_in_tpm,sc_sig_gene_names)

# Write labels for bDEA significant DEG's
for i, gene_name in enumerate(sc_sig_gene_names):

    if ~np.isnan(bDEA_p_norm[i]):
        
        # if bDEA_p_norm[i]>(-np.log(0.01)/np.nanmax(-np.log(sc_sig_p_from_bDEA))):

            sig_sc_bulk_DC = np.append(sig_sc_bulk_DC, gene_name)
            # plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)

        # Uncomment to print names of  most downregulated genes
        # if sc_sig_LFC_from_bDEA[i]<-0.5:

        #      plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)

        # To find individual genes
    
    if np.isin(gene_name,sig_in_tpm):    

        plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, weight='semibold',zorder=15)



# Mapping from bDEA p values to dot sizes
def size_mapping(value):
    return 2.+50.*(-np.log(value)/np.nanmax(-np.log(sc_sig_p_from_bDEA)))**2.

# Values you want to map to sizes
values = [0.0005, 0.005, 0.05]
sizes = [size_mapping(val) for val in values]


from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=f'{val:.0e}',
                          markerfacecolor='k', alpha=0.3, markersize=np.sqrt(size)) for val, size in zip(values, sizes)]

# dashed line above which significantly DE for cell type
ax.axhline(np.log(-np.log(0.05)), linestyle='--', linewidth=0.8, alpha=0.7)

# Create and add the legend to the plot
lg = ax.legend(handles=legend_elements, fontsize=6, loc='upper left')
lg.set_title(title=" Bulk DEA \n p-value for \n response", prop={'size':'small'})

plt.title('DEG\'s in DC\'s vs. rest in the monocyte cluster')
plt.xlabel(r'$\mathrm{log}_2$-fold change')
plt.ylabel('log(-log) of adjusted p-value')
# plt.legend()

plt.xlim(-7.3,8.9)
# plt.ylim(-1.25,5.9)

plt.savefig('Volcano_DCsubCl_vs_rest_in_monocytes_wilcoxon.pdf', bbox_inches='tight')

plt.show()

In [ ]:
# Of all plotted genes above, take only the ones with text label, i.e. sign. DEG's in bulk for response
# sig_bulk_mask = np.isin(sc_sig_gene_names,sig_in_bDEA_and_tpm)
sig_bulk_mask = np.isin(sc_sig_gene_names,sig_in_tpm)

# Order according to score in DEA for cell type
score_order = np.argsort(sc_sig_scores_ordered[sig_bulk_mask])[::-1]

data = np.zeros((4,len(sc_sig_gene_names[sig_bulk_mask])))
data[0,:] = sc_sig_scores_ordered[sig_bulk_mask][score_order]
data[1,:] = np.abs(sc_sig_scores_ordered[sig_bulk_mask])[score_order]
data[2,:] = sc_sig_p_from_bDEA[sig_bulk_mask][score_order]
data[3,:] = sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order]

df = pd.DataFrame(data=data.T, index = sc_sig_gene_names[sig_bulk_mask][score_order], columns= ['scScore', 'scAbsScore', 'bulk_p', 'bulkLFC'])

In [ ]:
from matplotlib import patches

In [ ]:
sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max()


In [ ]:
sc_sig_LFC_from_bDEA[sig_bulk_mask].min(), sc_sig_LFC_from_bDEA[sig_bulk_mask].max()

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(8,4))
axs[0].grid(False)
axs[1].grid(False)

fig.subplots_adjust(hspace=0)

# cmap_DC = plt.cm.cmap_DC_RdBu052
c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
# colors0=cmap_DC(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']>=0]))
colors0=cmap_DC(c_norm(df[df['scScore']>0]['bulkLFC']))

width0=0.5*np.ones(sum(df['scScore']>0))
centers0 = np.zeros(sum(df['scScore']>0))
for bar in np.arange(1,sum(df['scScore']>0)):
    
    centers0[bar] = centers0[bar-1]+width0[bar-1]/2.+width0[bar]/2.+0.25
    

rects0 = axs[0].bar(centers0, df[df['scScore']>0]['scScore'], width=width0, color=colors0, edgecolor='darkblue', linewidth=0.25, alpha=0.9)

i=0
for rect in rects0:

    gene = df[df['scScore']>0].index[i]
    bulk_p = round(df[df['scScore']>0]['bulk_p'],4)[i]
        
    height = rect.get_height()
    
    axs[0].text(rect.get_x() + rect.get_width()/2., height+0.75,
                gene,
                ha='center', va='bottom',rotation=90, fontsize=6)


    if bulk_p < 0.0001:

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+5,
                '****',
                ha='center', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+5,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+5,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+5,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
    i = i+1

sm0 = ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(0., sc_sig_LFC_from_bDEA.max()))

sm0.set_array([])

cbar0 = plt.colorbar(sm0, ax=axs[0], location='right', pad= 0.01)
cbar0.ax.tick_params(labelsize=7, rotation=90)
cbar0.ax.set_yticks([0,0.5,1.0,1.5])
cbar0.ax.set_yticklabels(['0.0','0.5','1.0','1.5'], fontsize=7, rotation=90)
# cbar.set_label(label='p-value for response in TPM data',size=9, labelpad=4)

axs[0].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[0].set_xticks([])
# axs[0].set_xlim(-1,34)

# ax.set_ylabel('Score', fontsize=10)
axs[0].set_yticks([20,15,10,5,0])
axs[0].set_ylim(0,23)


c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
# colors1=cmap_DC(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']<0]))
colors1=cmap_DC(c_norm(df[df['scScore']<0]['bulkLFC']))
width1= 0.5*np.ones(sum(df['scScore']<0))

centers1 = np.zeros(sum(df['scScore']<0))
for bar in np.arange(1,sum(df['scScore']<0)):
    
    centers1[bar] = centers1[bar-1]+width1[bar-1]/2.+width1[bar]/2.+0.25*(sum(df['scScore']>0)/sum(df['scScore']<0))


rects1 = axs[1].bar(centers1, df[df['scScore']<0]['scScore'], width=width1, color=colors1, edgecolor='darkblue', linewidth=0.25, alpha=0.9)
# rects1 = axs[1].bar(np.arange(sum(df['Score']<0)), df[df['Score']<0]['Score'], width, color=colors1)

# Write gene labels
i=0
for rect in rects1:

    gene = df[df['scScore']<0].index[i]
    height = rect.get_height()

    bulk_p = round(df[df['scScore']<0]['bulk_p'],4)[i]
        
    axs[1].text(rect.get_x() + rect.get_width()/2., height-0.6*len(gene)-2.5,
            gene,
            ha='center', va='bottom',rotation=90, fontsize=6)
    
    if bulk_p < 0.0001:

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.125, height-0.6*len(gene)- 4.5,
                '****',
                ha='left', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.125, height-0.6*len(gene)- 4.5,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[1].text(rect.get_x() + rect.get_width()/2., height-0.6*len(gene)- 4.5,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.125, height-0.6*len(gene)- 4.5,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
        
        # axs[1].text(rect.get_x() + rect.get_width()/2., height-0.5*len(gene)-0.4,
        #         gene +'  p='+str(bulk_p),
        #         ha='center', va='bottom',rotation=90, fontsize=6)
    
    
    
    i = i+1

sm1 = ScalarMappable(cmap=plt.cm.Reds_r, norm=plt.Normalize(0.,np.abs(sc_sig_LFC_from_bDEA.min())))
sm1.set_array([])

cbar1 = plt.colorbar(sm1, ax=axs[1], location='right', pad=0.01)
cbar1.ax.tick_params(labelsize=7)
cbar1.ax.set_yticks(-(sc_sig_LFC_from_bDEA.min()+np.array([1.5,1.0,0.5])))
cbar1.ax.set_yticklabels(['-1.5','-1.0','-0.5'], fontsize=7, rotation=90)
# cbar1.set_label(label='p-value for response in TPM data',size=9, labelpad=4)

axs[1].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[1].set_xticks([])
# axs[1].set_xlim(-1,40)

# axs[1].set_ylabel('Score', fontsize=10)
# axs[1].set_yticks([-10,-5])
# axs[1].set_ylim(-13,0)

# Use the same scale as for upregulated genes
axs[1].set_yticks([-20,-15,-10,-5,-0])
axs[1].set_ylim(-23,0)

# plt.title('scDEA of Select genes: DC subcluster vs other monocytes', fontsize=10)

plt.savefig('DC_mono_DEA_barplot_with_bulkDEA_p_asterisks.pdf',dpi=1200, bbox_inches='tight')

# add_width_legend(axs[0])

plt.show()


# For plotting color bar alone

fig, ax = plt.subplots(figsize = (4.,0.2))

cmap_cb = plt.cm.Blues
# norm = matplotlib.colors.Normalize(0, sc_sig_LFC_from_bDEA[sig_bulk_mask].max())
norm = matplotlib.colors.Normalize(0, 1.5)

# norm = plt.colors.Normalize(vmin = df.p_corr.min(), vmax =0.05)

mapper = cm.ScalarMappable(norm = norm, cmap = cmap_cb)

cbar = matplotlib.colorbar.ColorbarBase(ax, cmap = cmap_cb, norm = norm, orientation = 'horizontal',ticks=[0, 0.5, 1,01.5])
cbar.ax.set_xticklabels(['0.0', '0.5', '1.0','1.5'], rotation=0) 

plt.savefig('DC_mono_DEA_barplot_with_bulkDEA_p_asterisks_cbar.pdf',dpi=1200, bbox_inches='tight')

plt.show()

## T Cells: T_reg vs Rest in the T cell cluster

In [ ]:
# Define new cluster without NK cells

Tcell_cl_noNK = ['Regulatory T cells',
 'Tcm/Naive helper T cells',
 'Tem/Temra cytotoxic T cells',
 'Tem/Trm cytotoxic T cells',
 'Trm cytotoxic T cells']

adata_Tcells_noNK_pre = adata_th_naive[adata_th_naive.obs['majority_voting'].isin(Tcell_cl_noNK)]

# Define a categorical variable for a cell belonging to the T_reg
adata_Tcells_noNK_pre.obs['T_reg'] = adata_Tcells_noNK_pre.obs['majority_voting'].isin(['Regulatory T cells']).astype('category')

# Define also a categorical variable for cytotoxicity
adata_Tcells_noNK_pre.obs['cytotoxic'] = adata_Tcells_noNK_pre.obs['majority_voting'].isin(['Tem/Temra cytotoxic T cells',
 'Tem/Trm cytotoxic T cells',
 'Trm cytotoxic T cells']).astype('category')

In [ ]:
sc.tl.rank_genes_groups(adata_Tcells_noNK_pre, groupby ='T_reg', method='wilcoxon', key_added = "Treg_vs_other_Tcells_pre_wilcoxon")
sc.pl.rank_genes_groups(adata_Tcells_noNK_pre, n_genes=25, sharey=False, key="Treg_vs_other_Tcells_pre_wilcoxon", save='Treg_vs_other_Tcells_pre_wilcoxon.pdf')


In [ ]:
scDEA_res = adata_Tcells_noNK_pre.uns['Treg_vs_other_Tcells_pre_wilcoxon']
groups = scDEA_res['names'].dtype.names

group = 'True'
sc_pvals = scDEA_res['pvals_adj'][group]
sc_sig_genes_mask = (sc_pvals < 0.05)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# A few sc-DEG's are not part of the bulk-DEA 
np.setdiff1d(sc_sig_gene_names,np.intersect1d(sc_sig_gene_names,bDEA_res.index))

In [ ]:
adata_Tcells_noNK_pre

In [ ]:
de_markers = sc.get.rank_genes_groups_df(adata_Tcells_noNK_pre, group='True', key='Treg_vs_other_Tcells_pre_wilcoxon')
de_markers = de_markers[(de_markers.pvals_adj < 0.03) & (np.abs(de_markers.logfoldchanges) > 1.0)]
de_markers.to_csv('./data/scDEA_Treg_results_padj003_lfc1.csv', index=False)

In [ ]:
de_markers[0:10]

In [ ]:
#Volcano plot

## Uncomment these to define a new cmap
orig_cmap = matplotlib.cm.RdBu
cmap_Treg = shiftedColorMap(orig_cmap, start=0., midpoint=0.3, name='cmap_Treg_RdBu03')

sig_in_bDEA_and_tpm = np.intersect1d(orderedGenes[orderedGenes_p<0.05],bDEA_res[bDEA_res['pvalue']<0.05].index.values)
sig_in_tpm = orderedGenes[orderedGenes_p<0.05]


scDEA_res = adata_Tcells_noNK_pre.uns['Treg_vs_other_Tcells_pre_wilcoxon']
groups = scDEA_res['names'].dtype.names

# Select the group 
group = 'True'

# Get the scores, p-values and log fold changes
sc_scores = scDEA_res['scores'][group]
sc_pvals = scDEA_res['pvals_adj'][group]
sc_logfoldchanges = scDEA_res['logfoldchanges'][group]

# Show only genes that pass a significance threshold (e.g., adjusted p-value < 0.05)
sc_sig_genes_mask = (sc_pvals < 0.5)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# Restrict to genes included in both analyses
sc_sig_gene_names = np.intersect1d(sc_sig_gene_names, bDEA_res.index)

# Scores, p-values and LFCs in DEA for cell type
sc_sig_scores_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_pvals_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_logfoldchanges_ordered = np.zeros(len(sc_sig_gene_names))

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_scores_ordered[i] = sc_scores[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_pvals_ordered[i] = sc_pvals[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_logfoldchanges_ordered[i] = sc_logfoldchanges[[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]]


sc_sig_neg_log_pvals_ordered = -np.log(sc_sig_pvals_ordered)

sc_sig_p_from_bDEA = np.array([])
sc_sig_LFC_from_bDEA = np.array([])

#Collect names of genes that are significantly DE in both Sc and bulk data
sig_sc_bulk_DC = np.array([])

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_p_from_bDEA = np.append(sc_sig_p_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['padj'])
    sc_sig_LFC_from_bDEA = np.append(sc_sig_LFC_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['log2FoldChange'])


labeled_genes_Treg = np.intersect1d(sig_in_tpm,sc_sig_gene_names)

# Create the volcano plot
fig, ax = plt.subplots(figsize=(6, 4))

bDEA_p_norm = -np.log(sc_sig_p_from_bDEA)/np.nanmax(-np.log(sc_sig_p_from_bDEA))


# Define colors for scatter plot
dotcolors = sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]
ringcolors = (1+np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.max(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.+(1-np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.min(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.

# Scatter plot
ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), facecolor='none', c=ringcolors, cmap=cmap_Treg, s=4+52.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=10,alpha=0.9)
scatter = ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), c=dotcolors, cmap=cmap_Treg, s=2+50.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=15,alpha=0.9)
ax.grid(False)


# Plot colorbar
cbar = fig.colorbar(scatter, shrink=0.75, pad=0.02)
cbar.ax.tick_params(labelsize=7)
cbar.set_label(label='Bulk DEA '+r'$\mathrm{log}_2$-fold change for response',size=7, labelpad=-1)

# Write labels for bDEA significant DEG's
for i, gene_name in enumerate(sc_sig_gene_names):

    # if ~np.isnan(bDEA_p_norm[i]):
    #     # if DEA_p_norm[i]>np.quantile(DEA_p_norm[~np.isnan(DEA_p_norm)],0.9):
    #     if bDEA_p_norm[i]>-np.log(0.05)/np.nanmax(-np.log(sc_sig_p_from_bDEA)):

    #         sig_sc_bulk_Treg = np.append(sig_sc_bulk_Treg, gene_name)
    #         plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)

    #     # Uncomment to print names of  most downregulated genes
    #     # if sc_sig_LFC_from_bDEA[i]<-0.5:

    #     #      plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)


    if np.isin(gene_name,sig_in_tpm):

        plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, fontweight='semibold',zorder=15)


# Mapping from bDEA p values to dot sizes
def size_mapping(value):
    return 2.+50.*(-np.log(value)/np.nanmax(-np.log(sc_sig_p_from_bDEA)))**2.

# Values you want to map to sizes
values = [0.0005, 0.005, 0.05]

sizes = [size_mapping(val) for val in values]

from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=f'{val:.0e}',
                          markerfacecolor='k', alpha=0.3, markersize=np.sqrt(size)) for val, size in zip(values, sizes)]

# Create and add the legend to the plot
lg = ax.legend(handles=legend_elements, fontsize=6, loc='upper right')
lg.set_title(title=" Bulk DEA \n p-value for \n response", prop={'size':'small'})

ax.axhline(np.log(-np.log(0.05)), linestyle='--', linewidth=0.8, alpha=0.7)

plt.title('DEG\'s in T$_{\mathrm{reg}}$ vs. rest in the T cell cluster')
plt.xlabel(r'$\mathrm{log}_2$-fold change')
plt.ylabel(r'$\mathrm{log(-log)}$ of adjusted p-value')
# plt.legend()

plt.xlim(-3.9,4.9)

plt.savefig('Volcano_Treg_vs_other_Tcells_wilcoxon.pdf',dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
# Of all plotted genes above, take only the ones with text label, i.e. sign. DEG's in bulk for response
sig_bulk_mask = np.isin(sc_sig_gene_names,sig_in_tpm)

# Order according to score in DEA for cell type
score_order = np.argsort(sc_sig_scores_ordered[sig_bulk_mask])[::-1]

data = np.zeros((4,len(sc_sig_gene_names[sig_bulk_mask])))
data[0,:] = sc_sig_scores_ordered[sig_bulk_mask][score_order]
data[1,:] = np.abs(sc_sig_scores_ordered[sig_bulk_mask])[score_order]
data[2,:] = sc_sig_p_from_bDEA[sig_bulk_mask][score_order]
data[3,:] = sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order]

df = pd.DataFrame(data=data.T, index = sc_sig_gene_names[sig_bulk_mask][score_order], columns= ['scScore', 'scAbsScore', 'bulk_p', 'bulkLFC'])

In [ ]:
sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max()

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(8,4))
axs[0].grid(False)
axs[1].grid(False)

fig.subplots_adjust(hspace=0)

c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
colors0=cmap_Treg(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']>=0]))
# colors0=cmap_Treg(c_norm(df[df['scScore']>0]['bulkLFC']))

width0=0.5*np.ones(sum(df['scScore']>0))
centers0 = np.zeros(sum(df['scScore']>0))
for bar in np.arange(1,sum(df['scScore']>0)):
    
    centers0[bar] = centers0[bar-1]+width0[bar-1]/2.+width0[bar]/2.+0.25
    

rects0 = axs[0].bar(centers0, df[df['scScore']>0]['scScore'], width=width0, color=colors0, edgecolor='darkblue', linewidth=0.25, alpha=0.9)

i=0
for rect in rects0:

    gene = df[df['scScore']>0].index[i]
    bulk_p = round(df[df['scScore']>0]['bulk_p'],4)[i]
        
    height = rect.get_height()
    
    axs[0].text(rect.get_x() + rect.get_width()/2., height+0.75,
                gene,
                ha='center', va='bottom',rotation=90, fontsize=6)


    if bulk_p < 0.0001:

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+4,
                '****',
                ha='center', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+4,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+4,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.125, height+0.5*len(gene)+4,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
    i = i+1

sm0 = ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(0., sc_sig_LFC_from_bDEA.max()))

sm0.set_array([])

cbar0 = plt.colorbar(sm0, ax=axs[0], location='right', pad= 0.01)
cbar0.ax.tick_params(labelsize=7, rotation=90)
cbar0.ax.set_yticks([0,0.5,1.0,1.5])
cbar0.ax.set_yticklabels(['0.0','0.5','1.0','1.5'], fontsize=7, rotation=90)
axs[0].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[0].set_xticks([])
# axs[0].set_xlim(-1,34)

# ax.set_ylabel('Score', fontsize=10)
axs[0].set_yticks([20,15,10,5,0])
axs[0].set_ylim(0,23)


c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
colors1=cmap_Treg(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']<0]))
# colors1=cmap_Treg(df[df['scScore']<0]['bulkLFC'])
width1= 0.5*np.ones(sum(df['scScore']<0))

centers1 = np.zeros(sum(df['scScore']<0))
for bar in np.arange(1,sum(df['scScore']<0)):
    
    centers1[bar] = centers1[bar-1]+width1[bar-1]/2.+width1[bar]/2.+0.25*(sum(df['scScore']>0)/sum(df['scScore']<0))


rects1 = axs[1].bar(centers1, df[df['scScore']<0]['scScore'], width=width1, color=colors1, edgecolor='darkblue', linewidth=0.25, alpha=0.9)
# rects1 = axs[1].bar(np.arange(sum(df['Score']<0)), df[df['Score']<0]['Score'], width, color=colors1)

# Write gene labels
i=0
for rect in rects1:

    gene = df[df['scScore']<0].index[i]
    height = rect.get_height()

    bulk_p = round(df[df['scScore']<0]['bulk_p'],4)[i]
        
    axs[1].text(rect.get_x() + rect.get_width()/2.,  height-0.7*len(gene)-1.5,
            gene,
            ha='center', va='bottom',rotation=90, fontsize=6)
    
    if bulk_p < 0.0001:

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.1, height-0.7*len(gene)- 3.5,
                '****',
                ha='left', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.075, height-0.7*len(gene)- 3.5,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.075, height-0.7*len(gene)- 3.5,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.05, height-0.7*len(gene)- 3.5,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
        
        # axs[1].text(rect.get_x() + rect.get_width()/2., height-0.5*len(gene)-0.4,
        #         gene +'  p='+str(bulk_p),
        #         ha='center', va='bottom',rotation=90, fontsize=6)
    
    
    
    i = i+1

sm1 = ScalarMappable(cmap=plt.cm.Reds_r, norm=plt.Normalize(0.,np.abs(sc_sig_LFC_from_bDEA.min())))
sm1.set_array([])

cbar1 = plt.colorbar(sm1, ax=axs[1], location='right', pad=0.01)
cbar1.ax.tick_params(labelsize=7)
cbar1.ax.set_yticks(-(sc_sig_LFC_from_bDEA.min()+np.array([0.8,0.4])))
cbar1.ax.set_yticklabels(['-0.8','-0.4'], fontsize=7, rotation=90)
# # cbar1.set_label(label='p-value for response in TPM data',size=9, labelpad=4)

axs[1].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[1].set_xticks([])
# axs[1].set_xlim(-1,40)

# axs[1].set_ylabel('Score', fontsize=10)
# axs[1].set_yticks([-20,-15,-10,-5])
# axs[1].set_ylim(-22,0)

# ax.set_ylabel('Score', fontsize=10)
axs[1].set_yticks([-20,-15,-10,-5,0])
axs[1].set_ylim(-23,0)

# plt.title('scDEA of Select genes: DC subcluster vs other monocytes', fontsize=10)

plt.savefig('Treg_DEA_barplot_with_bulkDEA_p_asterisks.pdf',dpi=1200, bbox_inches='tight')

# add_width_legend(axs[0])

plt.show()



## Cytotoxic T cells vs naive and regulatory T cells

In [ ]:
sc.tl.rank_genes_groups(adata_Tcells_noNK_pre, groupby ='cytotoxic', method='wilcoxon', key_added = "cytotoxic_vs_other_Tcells_pre_wilcoxon")
sc.pl.rank_genes_groups(adata_Tcells_noNK_pre, n_genes=25, sharey=False, key="cytotoxic_vs_other_Tcells_pre_wilcoxon", save='cytotoxic_vs_other_Tcells_pre_wilcoxon.pdf')


In [ ]:
scDEA_res = adata_Tcells_noNK_pre.uns['cytotoxic_vs_other_Tcells_pre_wilcoxon']
groups = scDEA_res['names'].dtype.names

group = 'True'
sc_pvals = scDEA_res['pvals_adj'][group]
sc_sig_genes_mask = (sc_pvals < 0.05)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# A few sc-DEG's are not part of the bulk-DEA 
np.setdiff1d(sc_sig_gene_names,np.intersect1d(sc_sig_gene_names,bDEA_res.index))

In [ ]:
adata_Tcells_noNK_pre

In [ ]:
de_markers = sc.get.rank_genes_groups_df(adata_Tcells_noNK_pre, group='True', key='cytotoxic_vs_other_Tcells_pre_wilcoxon')
de_markers = de_markers[(de_markers.pvals_adj < 0.03) & (np.abs(de_markers.logfoldchanges) > 1.0)]
de_markers.to_csv('./data/scDEA_Tcyt_results_padj003_lfc1.csv', index=False)

In [ ]:
#Volcano plot

## Uncomment these to define a new cmap (1st time running this script)
orig_cmap = matplotlib.cm.RdBu
cmap_cytotoxic = shiftedColorMap(orig_cmap, start=0., midpoint=0.333, name='cmap_cytotoxic_RdBu0333')

scDEA_res = adata_Tcells_noNK_pre.uns['cytotoxic_vs_other_Tcells_pre_wilcoxon']
groups = scDEA_res['names'].dtype.names

# Select the group 
group = 'True'

sig_in_bDEA_and_tpm = np.intersect1d(orderedGenes[orderedGenes_p<0.05],bDEA_res[bDEA_res['pvalue']<0.05].index.values)
sig_in_tpm = orderedGenes[orderedGenes_p<0.05]

# Get the scores, p-values and log fold changes
sc_scores = scDEA_res['scores'][group]
sc_pvals = scDEA_res['pvals_adj'][group]
sc_logfoldchanges = scDEA_res['logfoldchanges'][group]

# Show only genes that pass a significance threshold (e.g., adjusted p-value < 0.05)
sc_sig_genes_mask = (sc_pvals < 0.5)

# Extract the names of the significant genes
sc_sig_gene_names = np.array(scDEA_res['names'][group])[sc_sig_genes_mask]

# Restrict to genes included in both analyses
sc_sig_gene_names = np.intersect1d(sc_sig_gene_names, bDEA_res.index)

# Scores, p-values and LFCs in DEA for cell type
sc_sig_scores_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_pvals_ordered = np.zeros(len(sc_sig_gene_names))
sc_sig_logfoldchanges_ordered = np.zeros(len(sc_sig_gene_names))

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_scores_ordered[i] = sc_scores[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_pvals_ordered[i] = sc_pvals[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]
    sc_sig_logfoldchanges_ordered[i] = sc_logfoldchanges[[np.where(scDEA_res['names'][group]==sc_sig_gene_names[i])[0]]]


sc_sig_neg_log_pvals_ordered = -np.log(sc_sig_pvals_ordered)

sc_sig_p_from_bDEA = np.array([])
sc_sig_LFC_from_bDEA = np.array([])

#Collect names of genes that are significantly DE in both Sc and bulk data
sig_sc_bulk_DC = np.array([])

for i in np.arange(len(sc_sig_gene_names)):

    sc_sig_p_from_bDEA = np.append(sc_sig_p_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['padj'])
    sc_sig_LFC_from_bDEA = np.append(sc_sig_LFC_from_bDEA, bDEA_res.loc[sc_sig_gene_names[i]]['log2FoldChange'])

labeled_genes_cytotoxicT = np.intersect1d(sig_in_tpm,sc_sig_gene_names)

# Create the volcano plot
fig, ax = plt.subplots(figsize=(6, 4))

bDEA_p_norm = -np.log(sc_sig_p_from_bDEA)/np.nanmax(-np.log(sc_sig_p_from_bDEA))

dotcolors = sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]
ringcolors = (1+np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.max(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.+(1-np.sign(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)]))*np.min(sc_sig_LFC_from_bDEA[~np.isnan(bDEA_p_norm)])/2.

ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), facecolor='none', c=ringcolors, cmap=cmap_cytotoxic, s=4+52.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=10,alpha=0.9)
scatter = ax.scatter(sc_sig_logfoldchanges_ordered[~np.isnan(bDEA_p_norm)], np.log(sc_sig_neg_log_pvals_ordered[~np.isnan(bDEA_p_norm)]), c=dotcolors, cmap=cmap_cytotoxic, s=2+50.*bDEA_p_norm[~np.isnan(bDEA_p_norm)]**2., zorder=15,alpha=0.9)
ax.grid(False)

# Plot colorbar
cbar = fig.colorbar(scatter, shrink=0.75, pad=0.02)
cbar.ax.tick_params(labelsize=7)
cbar.set_label(label='Bulk DEA '+r'$\mathrm{log}_2$-fold change for response',size=7, labelpad=0)

for i, gene_name in enumerate(sc_sig_gene_names):


    # if ~np.isnan(bDEA_p_norm[i]):
        
    #     if bDEA_p_norm[i]>-np.log(0.05)/np.nanmax(-np.log(sc_sig_p_from_bDEA)):

    #         sig_sc_bulk_CT = np.append(sig_sc_bulk_CT, gene_name)
    #         plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)

    #     # Uncomment to print names of  most downregulated genes
    #     # if sc_sig_LFC_from_bDEA[i]<-0.5:

    #     #      plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, zorder=15)


    if np.isin(gene_name,sig_in_tpm):

        plt.text(sc_sig_logfoldchanges_ordered[i]+0.1, np.log(sc_sig_neg_log_pvals_ordered[i])+0.05, gene_name, fontsize=6, fontweight='semibold', zorder=15)


def size_mapping(value):
    return 2.+50.*(-np.log(value)/np.nanmax(-np.log(sc_sig_p_from_bDEA)))**2.

# Values you want to map to sizes
values = [0.0005, 0.005, 0.05]
# values = [1, 15, 40]
sizes = [size_mapping(val) for val in values]

from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=f'{val:.0e}',
                          markerfacecolor='grey', alpha=1, markersize=np.sqrt(size)) for val, size in zip(values, sizes)]

# Create and add the legend to the plot
lg = ax.legend(handles=legend_elements, fontsize=6, loc='upper left')
lg.set_title(title=" Bulk DEA \n p-value for \n response", prop={'size':'small'})

ax.axhline(np.log(-np.log(0.05)), linestyle='--', linewidth=0.8, alpha=0.7)
plt.title('DEG\'s in cytotoxic T cells vs. naive and regulatory T cells')
plt.xlabel(r'$\mathrm{log}_2$-fold change')
plt.ylabel(r'$log(-log)$ of adjusted p-value')
# plt.legend()

# plt.xlim(-4.7,4.7)

plt.savefig('Volcano_cytotoxic_vs_other_Tcells_wilcoxon.pdf',dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
# Of all plotted genes above, take only the ones with text label, i.e. sign. DEG's in bulk for response
sig_bulk_mask = np.isin(sc_sig_gene_names,sig_in_tpm)

# Order according to score in DEA for cell type
score_order = np.argsort(sc_sig_scores_ordered[sig_bulk_mask])[::-1]

data = np.zeros((4,len(sc_sig_gene_names[sig_bulk_mask])))
data[0,:] = sc_sig_scores_ordered[sig_bulk_mask][score_order]
data[1,:] = np.abs(sc_sig_scores_ordered[sig_bulk_mask])[score_order]
data[2,:] = sc_sig_p_from_bDEA[sig_bulk_mask][score_order]
data[3,:] = sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order]

df = pd.DataFrame(data=data.T, index = sc_sig_gene_names[sig_bulk_mask][score_order], columns= ['scScore', 'scAbsScore', 'bulk_p', 'bulkLFC'])

In [ ]:
sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max()

In [ ]:
np.abs(sc_sig_LFC_from_bDEA.min())-0.5

In [ ]:
sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']<0]

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(8,4))
axs[0].grid(False)
axs[1].grid(False)

fig.subplots_adjust(hspace=0)

c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
colors0=cmap_cytotoxic(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']>=0]))

width0=0.5*np.ones(sum(df['scScore']>0))
centers0 = np.zeros(sum(df['scScore']>0))
for bar in np.arange(1,sum(df['scScore']>0)):
    
    centers0[bar] = centers0[bar-1]+width0[bar-1]/2.+width0[bar]/2.+0.25
    

rects0 = axs[0].bar(centers0, df[df['scScore']>0]['scScore'], width=width0, color=colors0, edgecolor='darkblue', linewidth=0.25, alpha=0.9)

i=0
for rect in rects0:

    gene = df[df['scScore']>0].index[i]
    bulk_p = round(df[df['scScore']>0]['bulk_p'],4)[i]
        
    height = rect.get_height()
    
    axs[0].text(rect.get_x() + rect.get_width()/2., height+2,
                gene,
                ha='center', va='bottom',rotation=90, fontsize=6)


    if bulk_p < 0.0001:

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.15, height+0.5*len(gene)+10,
                '****',
                ha='center', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.15, height+0.5*len(gene)+10,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.15, height+0.5*len(gene)+10,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[0].text(rect.get_x() + rect.get_width()/2.+0.15, height+0.5*len(gene)+10,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
    i = i+1

sm0 = ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(0., sc_sig_LFC_from_bDEA.max()))

sm0.set_array([])

cbar0 = plt.colorbar(sm0, ax=axs[0], location='right', pad= 0.01)
cbar0.ax.tick_params(labelsize=7, rotation=90)
cbar0.ax.set_yticks([0,0.5,1.0,1.5])
cbar0.ax.set_yticklabels(['0.0','0.5','1.0','1.5'], fontsize=7, rotation=90)

axs[0].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[0].set_xticks([])
# axs[0].set_xlim(-1,34)

# ax.set_ylabel('Score', fontsize=10)
axs[0].set_yticks([0,10,20,30])
axs[0].set_ylim(0,36)


# colors1=cmap_DC(df[df['scScore']<0]['bulkLFC'])

c_norm = plt.Normalize(sc_sig_LFC_from_bDEA.min(), sc_sig_LFC_from_bDEA.max())
colors1=cmap_cytotoxic(c_norm(sc_sig_LFC_from_bDEA[sig_bulk_mask][score_order][df['scScore']<0]))
width1= 0.5*np.ones(sum(df['scScore']<0))

centers1 = np.zeros(sum(df['scScore']<0))
for bar in np.arange(1,sum(df['scScore']<0)):
    
    centers1[bar] = centers1[bar-1]+width1[bar-1]/2.+width1[bar]/2.+0.25*(sum(df['scScore']>0)/sum(df['scScore']<0))


rects1 = axs[1].bar(centers1, df[df['scScore']<0]['scScore'], width=width1, color=colors1, edgecolor='darkblue', linewidth=0.25, alpha=0.9)
# rects1 = axs[1].bar(np.arange(sum(df['Score']<0)), df[df['Score']<0]['Score'], width, color=colors1)

# Write gene labels
i=0
for rect in rects1:

    gene = df[df['scScore']<0].index[i]
    height = rect.get_height()

    bulk_p = round(df[df['scScore']<0]['bulk_p'],4)[i]
        
    axs[1].text(rect.get_x() + rect.get_width()/2.,  height-len(gene)-3.0,
            gene,
            ha='center', va='bottom',rotation=90, fontsize=6)
    
    if bulk_p < 0.0001:

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.125, height-len(gene)- 6.0,
                '****',
                ha='left', va='center',rotation=90, fontsize=12)
     
    if ((bulk_p < 0.001)&(bulk_p >= 0.0001)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.075, height-len(gene)- 6.0,
                '***',
                ha='center', va='center',rotation=90, fontsize=12)
    
    if ((bulk_p < 0.01)&(bulk_p >= 0.001)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.075, height-len(gene)- 6.,
                '**',
                ha='center', va='center',rotation=90, fontsize=12)

    if ((bulk_p < 0.05)&(bulk_p >= 0.01)):

        axs[1].text(rect.get_x() + rect.get_width()/2.+0.075, height-len(gene)- 6.0,
                '*',
                ha='center', va='center',rotation=90, fontsize=12)
        
        # axs[1].text(rect.get_x() + rect.get_width()/2., height-0.5*len(gene)-0.4,
        #         gene +'  p='+str(bulk_p),
        #         ha='center', va='bottom',rotation=90, fontsize=6)
    
    
    
    i = i+1

sm1 = ScalarMappable(cmap=plt.cm.Reds_r, norm=plt.Normalize(0.,np.abs(sc_sig_LFC_from_bDEA.min())))
sm1.set_array([])

cbar1 = plt.colorbar(sm1, ax=axs[1], location='right', pad=0.01)
cbar1.ax.tick_params(labelsize=7)
cbar1.ax.set_yticks(-(sc_sig_LFC_from_bDEA.min()+np.array([1.0,0.5])))
cbar1.ax.set_yticklabels(['-1.0','-0.5'], fontsize=7, rotation=90)
# cbar1.set_label(label='p-value for response in TPM data',size=9, labelpad=4)

axs[1].tick_params(top=False, labeltop=True, bottom=False, labelbottom=False, rotation=90)
# ax.tight_layout()
axs[1].set_xticks([])
# axs[1].set_xlim(-1,40)

# axs[1].set_ylabel('Score', fontsize=10)
# axs[1].set_yticks([-20,-15,-10,-5])
# axs[1].set_ylim(-21,0)

axs[1].set_yticks([-30,-20,-10])
axs[1].set_ylim(-36,0)

# plt.title('scDEA of Select genes: DC subcluster vs other monocytes', fontsize=10)

plt.savefig('cytotoxicT_DEA_barplot_with_bulkDEA_p_asterisks.pdf',dpi=1200, bbox_inches='tight')

# add_width_legend(axs[0])

plt.show()

# fig, ax = plt.subplots(figsize = (0.3, 3.))

# cmap_cb = plt.cm.Blues
# norm = matplotlib.colors.Normalize(0, sc_sig_LFC_from_bDEA[sig_bulk_mask].max())
# # norm = matplotlib.colors.Normalize(0, 1.6)

# # norm = plt.colors.Normalize(vmin = df.p_corr.min(), vmax =0.05)

# mapper = cm.ScalarMappable(norm = norm, cmap = cmap_cb)

# cbl = matplotlib.colorbar.ColorbarBase(ax, cmap = cmap_cb, norm = norm, orientation = 'vertical')

# plt.show()

## Marimekko plot of expression level distribution 

In [ ]:
monocyte_cl = ['Erythrophagocytic macrophages','Macrophages', 'Intermediate macrophages', 'Suppressor macrophages',
               'pDC', 'DC', 'DC2','Migratory DCs',
               'Non-classical monocytes','Classical monocytes']

Tcell_cl_noNK = ['Regulatory T cells',
 'Tcm/Naive helper T cells',
 'Tem/Temra cytotoxic T cells',
 'Tem/Trm cytotoxic T cells',
 'Trm cytotoxic T cells']

In [ ]:
# Order cell types for a nice plot!
monocytes_Tcells_ordered = pd.Series(['Erythrophagocytic macrophages','Macrophages',
 'Intermediate macrophages',
 'Suppressor macrophages',
 'pDC',
 'DC',
 'DC2',
 'Migratory DCs',
 'Non-classical monocytes',
 'Classical monocytes','Regulatory T cells','Tcm/Naive helper T cells', 'Tem/Temra cytotoxic T cells',
    'Tem/Trm cytotoxic T cells', 'Trm cytotoxic T cells'], dtype="category")

In [ ]:
adata_monocytes_Tcells_pre = adata_th_naive[adata_th_naive.obs['majority_voting'].isin(monocytes_Tcells_ordered)]


In [ ]:
select_genes = np.unique(np.append(labeled_genes_DC,labeled_genes_Treg))
select_genes = np.unique(np.append(select_genes,labeled_genes_cytotoxicT))
select_genes = np.intersect1d(select_genes, adata_monocytes_Tcells_pre.var_names.values)

In [ ]:
adata_monocytes_Tcells_pre = adata_th_naive[adata_th_naive.obs['majority_voting'].isin(list(np.append(monocyte_cl, Tcell_cl_noNK)))]
adata_monocytes_Tcells_pre_sel = adata_monocytes_Tcells_pre[:,adata_monocytes_Tcells_pre.var_names.isin(select_genes)]

In [ ]:
mean_expression_Tcells  = np.asarray(adata_monocytes_Tcells_pre_sel[adata_monocytes_Tcells_pre_sel.obs['majority_voting'].isin(Tcell_cl_noNK)].X.mean(axis=0))[0]
mean_expression_monocytes = np.asarray(adata_monocytes_Tcells_pre_sel[adata_monocytes_Tcells_pre_sel.obs['majority_voting'].isin(monocyte_cl)].X.mean(axis=0))[0]

In [ ]:
exp_ratio = mean_expression_monocytes/(10.**(-16)+mean_expression_Tcells)
genes_sorted_by_expression = adata_monocytes_Tcells_pre_sel.var_names[exp_ratio.argsort()[::-1]]

In [ ]:
dp = sc.pl.dotplot(adata_monocytes_Tcells_pre_sel, var_names=genes_sorted_by_expression, groupby='majority_voting',  standard_scale='var',mean_only_expressed=False, figsize=(20,5), categories_order = monocytes_Tcells_ordered, var_group_labels=None,var_group_rotation=10, return_fig=True);
dp.legend(show=False)
dp.style(cmap='Blues')

dp.savefig('dotplot_monocytes_Tcells_pre_selected_genes.pdf', dpi=1200)

In [ ]:
dp = sc.pl.dotplot(adata_monocytes_Tcells_pre_sel, var_names=genes_sorted_by_expression, groupby='majority_voting', dendrogram=False,categories_order = monocytes_Tcells_ordered,
                   standard_scale='var', smallest_dot=30, color_map='Blues', figsize=(6,20), var_group_labels=False, swap_axes=True, return_fig=True)

dp.savefig('dotplot_monocytes_Tcells_pre_selected_genes.pdf', dpi=1200)
plt.show()